In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer


In [ ]:

y_train = train_data["Survived"] 

features = ["Pclass", "Sex", "Age", "SibSp", "Parch"] 
X_train = pd.get_dummies(train_data[features]) 
imputer = KNNImputer( n_neighbors=5)
X_train = imputer.fit_transform(X_train)

In [ ]:
X_test = pd.get_dummies(test_data[features])
imputer = KNNImputer( n_neighbors=5)
X_test = imputer.fit_transform(X_test)


maxdepth_space = np.arange(2,12, 1)
min_samples_split_space = np.arange(2,12, 1)
param_grid = {'max_depth': maxdepth_space, 
             'min_samples_split': min_samples_split_space}

rfc = RandomForestClassifier()
rfc_cv = GridSearchCV(rfc, param_grid, cv=5)

rfc_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(rfc_cv.best_params_)) 
print("Best score is {}".format(rfc_cv.best_score_))


In [ ]:
rfc = RandomForestClassifier(max_depth=rfc_cv.best_params_['max_depth'], min_samples_split=rfc_cv.best_params_['min_samples_split'])

model = rfc.fit(X_train, y_train)


In [ ]:
y_pred = rfc.predict(X_train)

In [ ]:
print("Accuracy: {}".format(accuracy_score(y_train, y_pred)))
print("f1:{}".format(f1_score(y_train, y_pred))) 

In [ ]:
predictions = rfc.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")